In [7]:
import os

import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms

import optuna
from optuna.trial import TrialState

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_class = 10
LOG_INTERVAL = 200

In [9]:
def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    
    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int(f"n_units_l{i}", 4, 128)
        layers.append(nn.Linear(in_features, out_features)) # (B, 1, 28, 28) => (B, 28 * 28) 
        layers.append(nn.ReLU())
        p = trial.suggest_float(f"dropout_l{i}", 0.2, 0.5)
        layers.append(nn.Dropout(p))
        
        in_features = out_features
    layers.append(nn.Linear(in_features, num_class))

    return nn.Sequential(*layers)

In [10]:
def get_dataloader(batch_size):
    train_dataset = datasets.FashionMNIST('./cifar10/', train=True, download=True, transform=transforms.ToTensor())
    valid_dataset = datasets.FashionMNIST('./cifar10/', train=False, transform=transforms.ToTensor())

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )
    
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )
    
    return train_loader, valid_loader

In [11]:
def objective(trial):
    model = define_model(trial).to(device)
    
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    epochs = trial.suggest_int("epoch", 5, 20)
    
    train_loader, valid_loader = get_dataloader(batch_size)
    
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        for batch_idx, (X_train, y_train) in enumerate(train_loader):
            X_train = X_train.view(X_train.size(0), -1).to(device) # flatten 역할
            y_train = y_train.to(device)

            optimizer.zero_grad()
            output = model(X_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()

            if batch_idx % LOG_INTERVAL == 0:
                print(f"Train Epoch: {epoch} [{(batch_idx + 1) * batch_size}/{len(train_loader.dataset)}]\tLoss: {loss.item():.6f}")
                
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (X_valid, y_valid) in enumerate(valid_loader):
                X_valid = X_valid.view(X_valid.size(0), -1).to(device)
                y_valid = y_valid.to(device)

                output = model(X_valid)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(y_valid.view_as(pred)).sum().item()
                
        accuracy = correct / len(valid_loader.dataset)

        trial.report(accuracy, epoch)

        if trial.should_prune(): # 잘라내야한다
            raise optuna.exceptions.TrialPruned()

        return accuracy

In [12]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600) # 100 개의 시도

# 
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-05-27 08:07:31,525] A new study created in memory with name: no-name-7bd8fcf3-b89c-4cf9-9839-8d55910694db


Train Epoch: 0 [64/60000]	Loss: 2.320259
Train Epoch: 0 [12864/60000]	Loss: 1.003493
Train Epoch: 0 [25664/60000]	Loss: 1.253901
Train Epoch: 0 [38464/60000]	Loss: 1.026822
Train Epoch: 0 [51264/60000]	Loss: 1.062552


[I 2021-05-27 08:07:37,373] Trial 0 finished with value: 0.7228 and parameters: {'n_layers': 3, 'n_units_l0': 124, 'dropout_l0': 0.33550063621548, 'n_units_l1': 87, 'dropout_l1': 0.48656991280952416, 'n_units_l2': 64, 'dropout_l2': 0.4302928111686385, 'optimizer': 'Adam', 'lr': 0.014910368946513042, 'batch_size': 64, 'epoch': 20}. Best is trial 0 with value: 0.7228.


Train Epoch: 0 [8/60000]	Loss: 2.345330
Train Epoch: 0 [1608/60000]	Loss: 1.042247
Train Epoch: 0 [3208/60000]	Loss: 0.638430
Train Epoch: 0 [4808/60000]	Loss: 2.379360
Train Epoch: 0 [6408/60000]	Loss: 0.980853
Train Epoch: 0 [8008/60000]	Loss: 0.641927
Train Epoch: 0 [9608/60000]	Loss: 0.641924
Train Epoch: 0 [11208/60000]	Loss: 0.554690
Train Epoch: 0 [12808/60000]	Loss: 0.209407
Train Epoch: 0 [14408/60000]	Loss: 0.539827
Train Epoch: 0 [16008/60000]	Loss: 0.340083
Train Epoch: 0 [17608/60000]	Loss: 0.390163
Train Epoch: 0 [19208/60000]	Loss: 0.076503
Train Epoch: 0 [20808/60000]	Loss: 0.733841
Train Epoch: 0 [22408/60000]	Loss: 0.622734
Train Epoch: 0 [24008/60000]	Loss: 0.717688
Train Epoch: 0 [25608/60000]	Loss: 0.503317
Train Epoch: 0 [27208/60000]	Loss: 1.407466
Train Epoch: 0 [28808/60000]	Loss: 0.563821
Train Epoch: 0 [30408/60000]	Loss: 0.353752
Train Epoch: 0 [32008/60000]	Loss: 0.453105
Train Epoch: 0 [33608/60000]	Loss: 0.313914
Train Epoch: 0 [35208/60000]	Loss: 0.12424

[I 2021-05-27 08:08:06,827] Trial 1 finished with value: 0.8152 and parameters: {'n_layers': 2, 'n_units_l0': 109, 'dropout_l0': 0.4149968283720924, 'n_units_l1': 59, 'dropout_l1': 0.3236666657692482, 'optimizer': 'Adam', 'lr': 0.0018213063504911014, 'batch_size': 8, 'epoch': 18}. Best is trial 1 with value: 0.8152.


...

Train Epoch: 0 [8/60000]	Loss: 2.298205
Train Epoch: 0 [1608/60000]	Loss: 1.320230
Train Epoch: 0 [3208/60000]	Loss: 1.294235
Train Epoch: 0 [4808/60000]	Loss: 0.924600
Train Epoch: 0 [6408/60000]	Loss: 1.646131
Train Epoch: 0 [8008/60000]	Loss: 1.390320
Train Epoch: 0 [9608/60000]	Loss: 0.785748
Train Epoch: 0 [11208/60000]	Loss: 0.882395
Train Epoch: 0 [12808/60000]	Loss: 0.593369
Train Epoch: 0 [14408/60000]	Loss: 0.810192
Train Epoch: 0 [16008/60000]	Loss: 1.005859
Train Epoch: 0 [17608/60000]	Loss: 0.965570
Train Epoch: 0 [19208/60000]	Loss: 0.929206
Train Epoch: 0 [20808/60000]	Loss: 0.548052
Train Epoch: 0 [22408/60000]	Loss: 0.907895
Train Epoch: 0 [24008/60000]	Loss: 0.856566
Train Epoch: 0 [25608/60000]	Loss: 1.237324
Train Epoch: 0 [27208/60000]	Loss: 0.833708
Train Epoch: 0 [28808/60000]	Loss: 1.094032
Train Epoch: 0 [30408/60000]	Loss: 1.333667
Train Epoch: 0 [32008/60000]	Loss: 0.732327
Train Epoch: 0 [33608/60000]	Loss: 0.988203
Train Epoch: 0 [35208/60000]	Loss: 0.22693

[I 2021-05-27 08:17:28,079] Trial 33 pruned. 


Train Epoch: 0 [8/60000]	Loss: 2.287897
Train Epoch: 0 [1608/60000]	Loss: 1.609064
Train Epoch: 0 [3208/60000]	Loss: 1.223159
Train Epoch: 0 [4808/60000]	Loss: 1.169493
Train Epoch: 0 [6408/60000]	Loss: 0.629120
Train Epoch: 0 [8008/60000]	Loss: 1.420549
Train Epoch: 0 [9608/60000]	Loss: 1.170269
Train Epoch: 0 [11208/60000]	Loss: 1.612399
Train Epoch: 0 [12808/60000]	Loss: 2.074721
Train Epoch: 0 [14408/60000]	Loss: 1.212459
Train Epoch: 0 [16008/60000]	Loss: 0.409768
Train Epoch: 0 [17608/60000]	Loss: 0.313329
Train Epoch: 0 [19208/60000]	Loss: 0.608351
Train Epoch: 0 [20808/60000]	Loss: 0.740305
Train Epoch: 0 [22408/60000]	Loss: 0.357406
Train Epoch: 0 [24008/60000]	Loss: 0.981329
Train Epoch: 0 [25608/60000]	Loss: 0.120731
Train Epoch: 0 [27208/60000]	Loss: 0.259260
Train Epoch: 0 [28808/60000]	Loss: 0.541450
Train Epoch: 0 [30408/60000]	Loss: 0.943284
Train Epoch: 0 [32008/60000]	Loss: 0.450106
Train Epoch: 0 [33608/60000]	Loss: 0.691698
Train Epoch: 0 [35208/60000]	Loss: 0.26256

[I 2021-05-27 08:18:03,613] Trial 34 finished with value: 0.8227 and parameters: {'n_layers': 3, 'n_units_l0': 109, 'dropout_l0': 0.2772047902118284, 'n_units_l1': 49, 'dropout_l1': 0.41556583571200606, 'n_units_l2': 96, 'dropout_l2': 0.3943526968351876, 'optimizer': 'Adam', 'lr': 0.0007356313362278379, 'batch_size': 8, 'epoch': 15}. Best is trial 6 with value: 0.8303.


Study statistics: 
  Number of finished trials:  35
  Number of pruned trials:  17
  Number of complete trials:  18
Best trial:
  Value:  0.8303
  Params: 
    n_layers: 1
    n_units_l0: 118
    dropout_l0: 0.43773201064875555
    optimizer: Adam
    lr: 0.001138316015475726
    batch_size: 64
    epoch: 19
